# Exercises in AST5210 using the RH code

Exercices from https://tiagopereira.space/ast5210/CaII_formation/.

---------------------

First off, we add a few heavier atoms to the `atoms.input` file according to the exercise description.

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d
from helita.vis import rh15d_vis
from shutil import copyfile

wl_ha = 656.28                            # nm (wavelength of H_alpha, from wikipedia)
wl_caII_H = 393.37                       # nm (wavelength of Ca II H)


# 'resets' files to originals from 'run_example' folder
copyfile("../og_atoms.input", "../atoms.input")
copyfile("../og_keyword.input", "../keyword.input");

In [2]:
# adjusts the 'atoms.inputs' file according to the exercise
atoms_file = open("../atoms.input", "r")
new_atoms_file = open("../new_atoms.input", "w")

change_nmetal = False
for line in atoms_file:
    if "# Nmetal" in line:
        change_nmetal = True
        new_atoms_file.write(line)
    elif change_nmetal:
        new_atoms_file.write("  10 \n")
        change_nmetal = False
    
    elif "H_6.atom" in line:
        H_line = line
        H_line = H_line.replace("ACTIVE", " PASSIVE")
        H_line = H_line.replace("pops.H.out", "")
        new_atoms_file.write(H_line)
    elif "CaII.atom" in line:
        # Removing this atom from file as it is re-added later
        #Ca_line = line
        #Ca_line = Ca_line.replace(" ACTIVE", "PASSIVE")
        #Ca_line = Ca_line.replace("pops.CA.out", "")
        #new_atoms_file.write(Ca_line)
        pass
    
    else:
        new_atoms_file.write(line)



# adding additional atoms to 'atoms.input' file
if "S.atom" not in line:
    new_atoms_file.write("  ../../Atoms/MgII-IRIS.atom       PASSIVE    ZERO_RADIATION   \n")
    new_atoms_file.write("  ../../Atoms/CaII_PRD.atom         ACTIVE    ZERO_RADIATION   \n")
    new_atoms_file.write("  ../../Atoms/Si.atom              PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/Al.atom              PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/Fe.atom              PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/He.atom              PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/N.atom               PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/Na.atom              PASSIVE   LTE_POPULATIONS   \n")
    new_atoms_file.write("  ../../Atoms/S.atom               PASSIVE   LTE_POPULATIONS   \n")

    

atoms_file.close()
new_atoms_file.close()

copyfile("../new_atoms.input", "../atoms.input")


# adjusts the 'keyword.inputs' file according to the exercise
keyword_file = open("../keyword.input", "r")
new_keyword_file = open("../new_keyword.input", "w")
for line in keyword_file:
    if "15D_WRITE_POPS" in line:
        new_keyword_file.write(line.replace("FALSE", "TRUE"))
    else:
        new_keyword_file.write(line)

keyword_file.close()
new_keyword_file.close()

copyfile("../new_keyword.input", "../keyword.input");

Creating the `ray.input` file containing indices of wavelengths to be saved. Most of the code in the next cell are taken from the exercise description.

In [4]:
try:
    data.close()
except:
    pass

data = rh15d.Rh15dout()
wave = data.ray.wavelength
indices = np.arange(len(wave))[(wave > 392.8) & (wave < 394.0)]

wave.sel(wavelength=500, method='nearest')
index500 = np.argmin(np.abs(wave.data - 500))

f = open('../ray.input', 'w')  # this will overwrite any existing file!
f.write('1.00\n')
output = str(len(indices) + 1)
for ind in indices:
    output += ' %i' % ind
output += ' %i\n' % index500 
f.write(output)
f.close()


--- Read ./output_aux.hdf5 file.
--- Read ./output_indata.hdf5 file.
--- Read ./output_ray.hdf5 file.


We then run the RH code and investigate the output from the `output_ray.hdf5` file using ncdump -h in the terminal. We get the following:

```
[jonastf@beehive17 run]$ ncdump -h output/output_ray.hdf5 
netcdf output_ray {
dimensions:
	height = 82 ;
	wavelength = 424 ;
	wavelength_selected = 44 ;
	x = 1 ;
	y = 1 ;
variables:
	float Jlambda(x, y, height, wavelength_selected) ;
		Jlambda:units = "W / (Hz m2 sr)" ;
		Jlambda:long_name = "Mean radiation field" ;
		Jlambda:_FillValue = 9.96921e+36f ;
	float chi(x, y, height, wavelength_selected) ;
		chi:units = "1 / m" ;
		chi:long_name = "Total absorption (line + continuum)" ;
		chi:_FillValue = 9.96921e+36f ;
	float intensity(x, y, wavelength) ;
		intensity:units = "W / (Hz m2 sr)" ;
		intensity:_FillValue = 9.96921e+36f ;
	float scattering(x, y, height, wavelength_selected) ;
		scattering:long_name = "Scattering term multiplied by Jlambda" ;
		scattering:_FillValue = 9.96921e+36f ;
	float source_function(x, y, height, wavelength_selected) ;
		source_function:units = "W / (Hz m2 sr)" ;
		source_function:long_name = "Total source function (line + continuum)" ;
		source_function:_FillValue = 9.96921e+36f ;
	double wavelength(wavelength) ;
		wavelength:units = "nm" ;
	int wavelength_indices(wavelength_selected) ;
	double wavelength_selected(wavelength_selected) ;
		wavelength_selected:units = "nm" ;
	double x(x) ;
		x:units = "m" ;
	double y(y) ;
		y:units = "m" ;

// global attributes:
		:atmosID = "FALC_82_5x5.hdf5 (Wed Jan 20 15:50:40 2021)" ;
		:snapshot_number = 0US ;
		:rev_id = "7d54c67 Jaime de la Cruz Rodriguez 2020-11-12 11:02:51 +0100" ;
		:nx = 1 ;
		:ny = 1 ;
		:nz = 82 ;
		:nwave = 424 ;
		:wavelength_selected = 44 ;
		:creation_time = "2021-02-11T16:12:40+0100" ;
}
```

We will then look further into the two arrays `chi` and `source function`. To find the optical depth, we need to integrate `chi` over height using functionalities that the `scipy` module provides. Code in the cell below is taken from the exercises.

In [5]:
from scipy.integrate import cumtrapz

height = data.atmos.height_scale[0, 0].dropna('height')                          # first column
index500 = np.argmin(np.abs(data.ray.wavelength_selected - 500))                 # index of 500 nm
tau500 = cumtrapz(data.ray.chi[0, 0, :, index500].dropna('height'), x=-height)
tau500 = np.concatenate([[1e-20], tau500])                                       # ensure tau500 and height have same size

unity_height = height[abs(tau500-1).argmin()].values/1e6                         # units of Mm

fig, ax = plt.subplots()
ax.plot(height / 1e6, tau500, label=r"$\tau_{500}$")  # height in Mm
ax.plot([np.min(height)/1e6, np.max(height)/1e6], [1,1], label=r"$\tau$ = 1")
ax.plot([unity_height, unity_height], [min(tau500), max(tau500)], label="Unity height (%.3f km)" % (unity_height*1e3))
ax.set_xlabel("Height (Mm)")
ax.set_ylabel(r"$\tau$")
ax.set_yscale("log")
ax.legend();

AttributeError: 'Dataset' object has no attribute 'wavelength_selected'

We observe that the unity heights at 500 nm is reached at approximately 1 km. 

In [27]:
index_CaII_H = np.argmin(np.abs(data.ray.wavelength_selected - wl_caII_H))  # index of 396.85 nm
tau_CaII_H = cumtrapz(data.ray.chi[0, 0, :, index_CaII_H].dropna('height'), x=-height)
tau_CaII_H = np.concatenate([[1e-20], tau_CaII_H])  # ensure tau_CaII_H and height have same size

unity_height = height[abs(tau_CaII_H-1).argmin()].values/1e6

fig_Ca, ax_Ca = plt.subplots()
ax_Ca.plot(height / 1e6, tau_CaII_H, label=r"$\tau_{Ca II H}$")  # height in Mm
ax_Ca.plot([np.min(height)/1e6, np.max(height)/1e6], [1,1], label=r"$\tau$ = 1")
ax_Ca.plot([unity_height, unity_height], [min(tau_CaII_H), max(tau_CaII_H)], label="Unity height (%.2f Mm)" % unity_height)
ax_Ca.set_xlabel("Height (Mm)")
ax_Ca.set_ylabel(r"$\tau$")
ax_Ca.set_yscale("log")
ax_Ca.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The unity height at the core of Ca II H occurs at approximately 1.7 Mm.

In [28]:
data.atom_CA

<xarray.Dataset>
Dimensions:          (height: 82, level: 6, phony_dim_4: 5, x: 1, y: 1)
Coordinates:
  * x                (x) float64 0.0
  * y                (y) float64 0.0
Dimensions without coordinates: height, level, phony_dim_4
Data variables:
    continuum        (phony_dim_4) uint32 ...
    line             (phony_dim_4) uint32 ...
    populations      (level, x, y, height) float32 ...
    populations_LTE  (level, x, y, height) float32 ...
Attributes:
    nlevel:      6
    nline:       5
    ncontinuum:  5

In [29]:
dep_coeff = (data.atom_CA.populations[0,0,0,:] / data.atom_CA.populations_LTE[0,0,0,:])[-len(height):]


fig_pop, (ax_pop1, ax_pop2) = plt.subplots(1, 2)
ax_pop1.plot(height, dep_coeff, label="Departure coefficient")
ax_pop1.set_xlabel("Height (Mm)")
ax_pop1.set_ylabel("Departure coefficient")
ax_pop1.legend()
ax_pop1.set_yscale("log")


ax_pop2.plot(tau500, dep_coeff, label="Departure coefficient")
ax_pop2.set_xlabel(r"$\tau_{500}$")
ax_pop2.set_ylabel("Departure coefficient")
ax_pop2.legend()
ax_pop2.set_xscale("log")
ax_pop2.set_yscale("log");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The left plot shows the departure coefficient as a function of height while the right plot shows the same but as a function of $\tau_{500}$.

In [59]:
height_tau1 = np.zeros(len(indices))

for i, w in enumerate(wave[indices]):
    index = np.argmin(np.abs(data.ray.wavelength_selected - w))
    tau = cumtrapz(data.ray.chi[0, 0, :, index].dropna('height'), x=-height)
    height_tau1[i] = height[np.argmin(abs(tau-1))]

height_tau1 /= 1e6
    
fig_tau1, ax_tau1 = plt.subplots()
ax_tau1.plot(wave[indices], height_tau1, label=r"$\tau$ = 1")
ax_tau1.plot([wl_caII_H, wl_caII_H], [np.min(height_tau1), np.max(height_tau1)], label="Ca II Core")
ax_tau1.legend()
ax_tau1.set_xlabel("Wavelength (nm)")
ax_tau1.set_ylabel("Height (Mm)");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The plot above shows where the optical depth is equal to one as a function of wavelegth. We note the highest point in the atmosphere presents the Ca II core.

We will look into the source funtion widget that `helita` provides.

In [60]:
rh15d_vis.SourceFunction(data);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='wavelength', max=43), Checkbox(value=True, description='…

We remember that the line core is at approximately 393.37 nm, i.e. index 21 in this case. The far wing can be repesented as either the first or the last index. Looking at index 0 shows us that the source function departs from the Planc function at approximately 0.6 Mm. This is also true at the line core but the emissivity follows the source function to approximately 1.8 Mm or $\tau$ = 1.